In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import seaborn
import json
from pandas.io.json import json_normalize

In [2]:
with open("language1.json") as file:
    json_ext = json.load(file)

In [161]:
import re
languages = ["Español", "Čeština", "Deutsch", "Français", "हिन्दी", "Bahasa Indonesia", "Italiano", "日本語", "Nederlands", "Português", "Русский", "العربية", "ไทย", "Türkçe", "Tiếng Việt",  "한국어", "中文"]
output = []
def nested_parser(json_obj, path=[]):
    path.append(json_obj["name"])
    if len(json_obj["children"]) == 0:
        for j in json_obj["content"]:
            try:
                content = j["meta"]["info_section"]["info_content"]
                ind = content.find("In other languages:")
                if ind!=-1:
                    langs = content.replace("\n\t\t\t\t\t\t\t ", "\xa0").split("\xa0")[1::2]
                    langs = [i[:-1] for i in langs]
                    langs = list(set(langs).intersection(set(languages)))
                    output.append((path,path[1], path[2],path[3], j["name"], j["url"], langs ))
            except:
                content = None
            
    for i in json_obj["children"]:
        nested_parser(i, path[:])
        

In [162]:
nested_parser(json_ext)

In [163]:
output[0]

(['Root',
  'Hobbies and Crafts',
  'Games',
  'Video Games',
  'Multi Platform Games',
  'Halo',
  'Halo 2'],
 'Hobbies and Crafts',
 'Games',
 'Video Games',
 'Get Halo 2 for Your PC',
 'https://www.wikihow.com/Get-Halo-2-for-Your-PC',
 ['Deutsch',
  'Русский',
  '中文',
  'Português',
  'Italiano',
  'Español',
  'Français',
  'Bahasa Indonesia'])

In [164]:
df = pd.DataFrame(output, columns=["path", "1", "2", "3", "name", "url", "languages"])

In [165]:
df.head()

,path,1,2,3,name,url,languages
0,"[Root, Hobbies and Crafts, Games, Video Games,...",Hobbies and Crafts,Games,Video Games,Get Halo 2 for Your PC,https://www.wikihow.com/Get-Halo-2-for-Your-PC,"[Deutsch, Русский, 中文, Português, Italiano, Es..."
1,"[Root, Hobbies and Crafts, Games, Video Games,...",Hobbies and Crafts,Games,Video Games,Play Halo 2,https://www.wikihow.com/Play-Halo-2,[Español]
2,"[Root, Hobbies and Crafts, Games, Video Games,...",Hobbies and Crafts,Games,Video Games,Glitch out of Maps in Halo 2,https://www.wikihow.com/Glitch-out-of-Maps-in-...,[Español]
3,"[Root, Hobbies and Crafts, Games, Video Games,...",Hobbies and Crafts,Games,Video Games,Be a Good Sniper in Black Ops 2,https://www.wikihow.com/Be-a-Good-Sniper-in-Bl...,"[Italiano, Español, Русский, Português]"
4,"[Root, Hobbies and Crafts, Games, Video Games,...",Hobbies and Crafts,Games,Video Games,Get Better at Zombies in Black Ops 2,https://www.wikihow.com/Get-Better-at-Zombies-...,"[Italiano, Español, Русский, Português]"


In [166]:
df.to_csv("languages_parsed.csv")

In [167]:
df.shape

(24539, 7)

In [169]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=languages)
one_hot_langs = mlb.fit_transform(df["languages"])


In [183]:
one_hot = pd.concat([df, pd.DataFrame(one_hot_langs, columns = languages)], axis=1).drop("languages", axis=1)

In [9]:
# one_hot.to_csv("one_hot.csv")
one_hot = pd.read_csv("one_hot.csv")

In [10]:
one_hot.head()

,path,1,2,3,name,url,Español,Čeština,Deutsch,Français,...,日本語,Nederlands,Português,Русский,العربية,ไทย,Türkçe,Tiếng Việt,한국어,中文
0,"['Root', 'Hobbies and Crafts', 'Games', 'Video...",Hobbies and Crafts,Games,Video Games,Get Halo 2 for Your PC,https://www.wikihow.com/Get-Halo-2-for-Your-PC,1,0,1,1,...,0,0,1,1,0,0,0,0,0,1
1,"['Root', 'Hobbies and Crafts', 'Games', 'Video...",Hobbies and Crafts,Games,Video Games,Play Halo 2,https://www.wikihow.com/Play-Halo-2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"['Root', 'Hobbies and Crafts', 'Games', 'Video...",Hobbies and Crafts,Games,Video Games,Glitch out of Maps in Halo 2,https://www.wikihow.com/Glitch-out-of-Maps-in-...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"['Root', 'Hobbies and Crafts', 'Games', 'Video...",Hobbies and Crafts,Games,Video Games,Be a Good Sniper in Black Ops 2,https://www.wikihow.com/Be-a-Good-Sniper-in-Bl...,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0
4,"['Root', 'Hobbies and Crafts', 'Games', 'Video...",Hobbies and Crafts,Games,Video Games,Get Better at Zombies in Black Ops 2,https://www.wikihow.com/Get-Better-at-Zombies-...,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0


In [15]:
one_hot.groupby("1").mean()

,Español,Čeština,Deutsch,Français,हिन्दी,Bahasa Indonesia,Italiano,日本語,Nederlands,Português,Русский,العربية,ไทย,Türkçe,Tiếng Việt,한국어,中文
1,,,,,,,,,,,,,,,,,
Arts and Entertainment,0.947649,0.055901,0.482697,0.419698,0.023070,0.329193,0.679681,0.033718,0.147294,0.847382,0.710736,0.103815,0.065661,0.005324,0.061224,0.031943,0.149956
Cars & Other Vehicles,0.928105,0.065359,0.359477,0.379085,0.017429,0.261438,0.714597,0.054466,0.137255,0.688453,0.808279,0.095861,0.047930,0.002179,0.032680,0.026144,0.080610
Computers and Electronics,0.954386,0.066127,0.592173,0.485290,0.079622,0.450202,0.760594,0.052901,0.250202,0.895547,0.771930,0.208367,0.162753,0.014305,0.142240,0.065857,0.252632
Education and Communications,0.940727,0.089866,0.579828,0.538719,0.062620,0.464149,0.735182,0.053059,0.292065,0.867113,0.730402,0.229924,0.138145,0.014818,0.148662,0.098470,0.235182
Family Life,0.956621,0.036530,0.465753,0.401826,0.025114,0.328767,0.614155,0.025114,0.100457,0.821918,0.746575,0.136986,0.034247,0.009132,0.107306,0.027397,0.102740
Finance and Business,0.936149,0.024558,0.357564,0.298625,0.014735,0.271120,0.492141,0.019646,0.042240,0.611984,0.584479,0.074656,0.023576,0.001965,0.049116,0.027505,0.103143
Health,0.965083,0.174898,0.643599,0.634476,0.072035,0.530670,0.785467,0.096571,0.347594,0.903429,0.773828,0.391003,0.206354,0.020447,0.296949,0.210129,0.245675
Hobbies and Crafts,0.918403,0.029080,0.475694,0.352865,0.017795,0.317708,0.619792,0.019531,0.155816,0.803385,0.711372,0.081163,0.045573,0.001302,0.033420,0.015191,0.123698
Holidays and Traditions,0.982456,0.043860,0.447368,0.421053,0.026316,0.298246,0.710526,0.008772,0.131579,0.842105,0.815789,0.105263,0.035088,0.000000,0.035088,0.017544,0.210526


In [12]:
one_hot.groupby("2").mean()

,Español,Čeština,Deutsch,Français,हिन्दी,Bahasa Indonesia,Italiano,日本語,Nederlands,Português,Русский,العربية,ไทย,Türkçe,Tiếng Việt,한국어,中文
2,,,,,,,,,,,,,,,,,
Aging Family,0.900000,0.000000,0.100000,0.000000,0.000000,0.000000,0.300000,0.000000,0.000000,0.600000,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Alternative Health,0.945148,0.101266,0.518987,0.417722,0.042194,0.354430,0.780591,0.092827,0.227848,0.873418,0.818565,0.219409,0.105485,0.016878,0.143460,0.109705,0.151899
Amphibian,0.954545,0.000000,0.545455,0.500000,0.000000,0.363636,0.681818,0.045455,0.272727,0.863636,0.772727,0.181818,0.000000,0.000000,0.045455,0.000000,0.181818
Amusement and Theme Parks,1.000000,0.333333,0.666667,0.666667,0.000000,0.666667,1.000000,0.000000,0.333333,1.000000,1.000000,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000
Animal Behavior,0.944444,0.055556,0.500000,0.500000,0.000000,0.500000,0.333333,0.000000,0.388889,0.666667,0.722222,0.055556,0.111111,0.000000,0.055556,0.055556,0.111111
Animal Rescue,1.000000,0.166667,0.666667,0.500000,0.000000,0.500000,1.000000,0.166667,0.333333,0.833333,0.833333,0.333333,0.333333,0.000000,0.333333,0.333333,0.166667
Artwork,1.000000,0.055556,0.500000,0.333333,0.000000,0.277778,0.666667,0.000000,0.055556,0.722222,0.777778,0.055556,0.055556,0.000000,0.000000,0.000000,0.166667
Aviation,0.937500,0.000000,0.250000,0.250000,0.000000,0.187500,0.562500,0.000000,0.000000,0.687500,0.687500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Banks and Financial Institutions,0.946237,0.010753,0.387097,0.279570,0.010753,0.236559,0.430108,0.010753,0.043011,0.440860,0.440860,0.075269,0.021505,0.000000,0.086022,0.032258,0.129032


In [27]:
one_hot.loc[:,"Español":"中文"].mean().sort_values(ascending=False)

Español             0.953380
Português           0.850075
Русский             0.743796
Italiano            0.707486
Deutsch             0.535922
Français            0.489466
Bahasa Indonesia    0.400261
Nederlands          0.232283
العربية             0.203105
中文                  0.189250
Tiếng Việt          0.130079
ไทย                 0.114186
Čeština             0.088145
한국어                 0.074005
日本語                 0.051632
हिन्दी              0.046090
Türkçe              0.009699
dtype: float64